In [1]:
import pandas as pd

lectures = pd.read_csv('/Users/bessonv/Downloads/defectdojo_api-master/venv/data/lectures.csv', sep = ',')
questions = pd.read_csv('/Users/bessonv/Downloads/defectdojo_api-master/venv/data/questions.csv', sep = ',')
train = pd.read_csv('/Users/bessonv/Downloads/defectdojo_api-master/venv/data/train.csv', sep = ',', index_col='row_id')

In [2]:
# Объединение с `questions` для получения данных о вопросах
train = train.merge(questions, left_on='content_id', right_on='question_id', how='left')

In [3]:
# Объединение с `lectures` для получения данных о лекциях
train = train.merge(lectures, left_on='content_id', right_on='lecture_id', how='left')

# Lectures

In [7]:
display(lectures.head())

,lecture_id,tag,part,type_of
0,89,159,5,concept
1,100,70,1,concept
2,185,45,6,concept
3,192,79,5,solving question
4,317,156,5,solving question


In [24]:
lectures.describe()

,lecture_id,tag,part
count,418.000000,418.000000,418.000000
mean,16983.401914,94.480861,4.267943
std,9426.164660,53.586487,1.872424
min,89.000000,0.000000,1.000000
25%,9026.250000,50.250000,2.000000
50%,17161.500000,94.500000,5.000000
75%,24906.250000,140.000000,6.000000
max,32736.000000,187.000000,7.000000


# Questions

In [6]:
display(questions.head())

,question_id,bundle_id,correct_answer,part,tags
0,0,0,0,1,51 131 162 38
1,1,1,1,1,131 36 81
2,2,2,0,1,131 101 162 92
3,3,3,0,1,131 149 162 29
4,4,4,3,1,131 5 162 38


In [26]:
questions.describe()

,question_id,bundle_id,correct_answer,part
count,13523.00000,13523.000000,13523.000000,13523.000000
mean,6761.00000,6760.510907,1.455298,4.264956
std,3903.89818,3903.857783,1.149707,1.652553
min,0.00000,0.000000,0.000000,1.000000
25%,3380.50000,3379.500000,0.000000,3.000000
50%,6761.00000,6761.000000,1.000000,5.000000
75%,10141.50000,10140.000000,3.000000,5.000000
max,13522.00000,13522.000000,3.000000,7.000000


# Train

In [4]:
display(train.head())

,timestamp,user_id,content_id,content_type_id,task_container_id,user_answer,answered_correctly,prior_question_elapsed_time,prior_question_had_explanation
row_id,,,,,,,,,
0,0,115,5692,0,1,3,1,NaN,NaN
1,56943,115,5716,0,2,2,1,37000.0,False
2,118363,115,128,0,0,0,1,55000.0,False
3,131167,115,7860,0,3,0,1,19000.0,False
4,137965,115,7922,0,4,1,1,11000.0,False


## Очистка данных

In [78]:
# Замена -1 на NaN
train['user_answer'] = train['user_answer'].replace(-1, pd.NA)
train['answered_correctly'] = train['answered_correctly'].replace(-1, pd.NA)

# Замена строковых значений на категориальные
train['prior_question_had_explanation'] = train['prior_question_had_explanation'].astype('category')

In [79]:
train.describe()

,timestamp,user_id,content_id,content_type_id,task_container_id,prior_question_elapsed_time
count,1.012303e+08,1.012303e+08,1.012303e+08,1.012303e+08,1.012303e+08,9.887879e+07
mean,7.703644e+09,1.076732e+09,5.219605e+03,1.935222e-02,9.040624e+02,2.542381e+04
std,1.159266e+10,6.197163e+08,3.866359e+03,1.377596e-01,1.358302e+03,1.994815e+04
min,0.000000e+00,1.150000e+02,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,5.243436e+08,5.408116e+08,2.063000e+03,0.000000e+00,1.040000e+02,1.600000e+04
50%,2.674234e+09,1.071781e+09,5.026000e+03,0.000000e+00,3.820000e+02,2.100000e+04
75%,9.924551e+09,1.615742e+09,7.425000e+03,0.000000e+00,1.094000e+03,2.966600e+04
max,8.742577e+10,2.147483e+09,3.273600e+04,1.000000e+00,9.999000e+03,3.000000e+05


## Cреднее время, которое тратили на вопрос студенты

In [80]:
train.groupby('answered_correctly')['prior_question_elapsed_time'].mean().reset_index(name='mean_prior_question_elapsed_time').round()

,answered_correctly,mean_prior_question_elapsed_time
0,0,25642.0
1,1,25310.0


Делаем вывод, что студенты отвечали на вопросы примерно одинаковое количество времени.

Можно добавить, что время затраченное на ответ не влияет на правильность ответа студента.

## Процентное соотношение правильных ответов на вопросы, где есть объяснение (True), и где его нет (False).

In [53]:
# Фильтрация ответов, где answered_correctly == 1
correct_answers = train.loc[train['answered_correctly'] == 1].groupby('prior_question_had_explanation')['answered_correctly'].size()

# Общее количество ответов в каждой группе prior_question_had_explanation
total_answers = train.groupby('prior_question_had_explanation')['answered_correctly'].size()

# Выражаем результат в процентах
percent_correct = (correct_answers / total_answers) * 100

percent_correct_df = percent_correct.reset_index(name='answered_correctly_percentage').round()

# Вывод результата
display(percent_correct_df)

,prior_question_had_explanation,answered_correctly_percentage
0,False,41.0
1,True,67.0


В ответах с объяснением правильных решений – 67%, а в ответах без объяснений правильных решений всего 41%.

*Вывод:* Учеников ответевших правильно с пояснением к решению – большинство.

# Пользовательские показатели

In [76]:
# Группировка по пользователям
user_performance = train.groupby('user_id')['answered_correctly'].mean()

# Отбор лучших пользователей (успеваемость выше среднего)
above_average_users = user_performance[user_performance > user_performance.mean()].count()

print(f'Учеников с успеваемостью выше среднего {round(float((above_average_users/user_performance.count())*100),2)}%')

Учеников с успеваемостью выше среднего 54.57%


## Объеденение таблиц

In [2]:
# Объединение с `questions` для получения данных о вопросах
train_and_questions = train.merge(questions, left_on='content_id', right_on='question_id', how='left')

# Объединение с `lectures` для получения данных о лекциях
train_and_lectures = train.merge(lectures, left_on='content_id', right_on='lecture_id', how='left')

# Часть теста (part)

In [18]:
# Успеваемость по частям теста
part_performance = train_and_questions.groupby('part')['answered_correctly'].mean().reset_index(name = 'answered_correctly_percentage')
part_performance['answered_correctly_percentage'] = part_performance['answered_correctly_percentage'] * 100
display(part_performance)

,part,answered_correctly_percentage
0,1.0,72.219824
1,2.0,70.191243
2,3.0,68.632123
3,4.0,60.914893
4,5.0,59.869141
5,6.0,65.488133
6,7.0,65.195551


In [19]:
part_performance.describe()

,part,answered_correctly_percentage
count,7.000000,7.000000
mean,4.000000,66.072987
std,2.160247,4.609744
min,1.000000,59.869141
25%,2.500000,63.055222
50%,4.000000,65.488133
75%,5.500000,69.411683
max,7.000000,72.219824


Вывод: Максимальное количество ответов в 1 части, минимальное в 5 части. В среднем, правильно ответили большинство.

In [9]:
# Разделение тегов и расчет средних значений
train['tags'] = train['tags'].fillna('').astype(str)
tag_performance = train.groupby('tags')['answered_correctly'].mean().reset_index(name = 'answered_correctly_percentage')
tag_performance['answered_correctly_percentage'] = tag_performance['answered_correctly_percentage'] * 100
display(tag_performance)

,tags,answered_correctly_percentage
0,,-99.999832
1,1,57.936679
2,1 162,62.280967
3,10 111 92,83.750696
4,10 164 102,76.147620
...,...,...
1515,98 97 21,72.081364
1516,98 97 35 117 122 162,71.819186
1517,98 97 35 146 122,71.699622
1518,98 97 46 135,68.493941
